In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
import warnings
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv('/kaggle/input/stoxx-2014-2024/STOXX - 2014-2024.csv')
data = data.drop(columns='Close')

In [ ]:
data.head()

# Identifying and Handling outliers

In [ ]:
def handling_outliers(df):
    outliers_info = {}
    for column in df.select_dtypes(include = ['number']).columns.to_list():
        mean = df[column].mean()
        std = df[column].std()
        outliers = df[(df[column] < mean-3*std) | (df[column] > mean+3*std)]
        outliers_info[column] = len(outliers)
        median = df[column].median()
        df.loc[(df[column] < mean-3*std) | (df[column] > mean+3*std), column] = median
    return outliers_info, df


outliers_info, data = handling_outliers(data)

In [ ]:
outliers_info

# Handling missing values

In [ ]:
missing_values = data.isnull().sum().sum()
missing_values

In [ ]:
data.interpolate(method='linear', inplace=True)

In [ ]:
data.isnull().sum().sum()

# ARIMA 

## Initial Analysis

In [ ]:
data['Date'] = pd.to_datetime(data['Date'])

In [ ]:
plt.plot(data['Date'], data['Adj Close'])
plt.xticks(rotation=45)
plt.xlabel('Time')
plt.ylabel('Adjusted Close price')
plt.title('STOXX Europe 600 Adjusted close price (2014 - 2024)')
plt.show()

In [ ]:
plot_acf(data['Adj Close'], lags=40, alpha=0.05)
plt.xlabel('Lags')
plt.ylabel('Autocorrelation')
plt.title('Autocorrelation Function')
plt.ylim(-0.1, 1.1)
plt.show()

In [ ]:
plot_pacf(data['Adj Close'], lags=40, alpha=0.05)
plt.xlabel('Lags')
plt.ylabel('Autocorrelation')
plt.title('Partial Autocorrelation Function')
plt.ylim(-0.1, 1.1)
plt.show()

In [ ]:
ad_full_result = adfuller(data['Adj Close'])
adf_results = pd.DataFrame({
    'Metric': [
        'ADF Statistic',
        'p-value',
        'Number of lags used',
        'Number of observations used'
    ],
    'Value': [
        round(ad_full_result[0], 3),
        round(ad_full_result[1], 3),
        ad_full_result[2],
        ad_full_result[3]
    ]
})

adf_results

## First Differencing

In [ ]:
data.set_index('Date', inplace=True)
stoxx_diff = data['Adj Close'].diff()

In [ ]:
plt.plot(stoxx_diff)
plt.xlabel('Time')
plt.ylabel('STOXX 600')
plt.title('First-differenced STOXX Europe 600')

In [ ]:
stoxx_diff = stoxx_diff.dropna()

In [ ]:
ad_full_result_diff = adfuller(stoxx_diff)
adf_results_diff = pd.DataFrame({
    'Metric': [
        'ADF Statistic',
        'p-value',
        'Number of lags used',
        'Number of observations used'
    ],
    'Value': [
        round(ad_full_result_diff[0], 3),
        round(ad_full_result_diff[1], 3),
        ad_full_result_diff[2],
        ad_full_result_diff[3]
    ]
})

adf_results_diff

In [ ]:
plot_acf(stoxx_diff, lags=40, alpha=0.05)
plt.xlabel('Lags')
plt.ylabel('Autocorrelation')
plt.title('Autocorrelation Function')
plt.ylim(-0.1, 1.1)
plt.show()

In [ ]:
plot_pacf(stoxx_diff, lags=40, alpha=0.05)
plt.xlabel('Lags')
plt.ylabel('Autocorrelation')
plt.title('Autocorrelation Function')
plt.ylim(-0.1, 1.1)
plt.show()

## Model selection

In [ ]:
warnings.filterwarnings('ignore')

p_range = range(0, 6)
d_range = [0, 1]
q_range = range (0, 6)

series = data['Adj Close']

train_size = int(len(series)*0.8)
train_series = series[:train_size]
test_series = series[train_size:]

result = []

for p in p_range:
    for d in d_range:
        for q in q_range:
            try:
                model = ARIMA(train_series, order=(p,d,q))
                res = model.fit()
    
                BIC = res.bic
    
                result.append({
                    'p': p,
                    'd': d,
                    'q': q,
                    'BIC': BIC
                })
            except Exception as e:
                print(e)
                continue

result_df = pd.DataFrame(result)
sorted_results = result_df.sort_values(by='BIC')

print(sorted_results)

In [ ]:
warnings.filterwarnings('ignore')

p_range = range(0, 6)
d_range = [0, 1]
q_range = range (0, 6)

series = data['Adj Close']

train_size = int(len(series)*0.8)
train_series = series[:train_size]
test_series = series[train_size:]

result = []

for p in p_range:
    for d in d_range:
        for q in q_range:
            try:
                model = ARIMA(train_series, order=(p,d,q))
                res = model.fit()
    
                AIC = res.aic
    
                result.append({
                    'p': p,
                    'd': d,
                    'q': q,
                    'AIC': AIC
                })
            except Exception as e:
                print(e)
                continue

result_df = pd.DataFrame(result)
sorted_results = result_df.sort_values(by='AIC')

print(sorted_results)

## Residuals check

In [ ]:
# Best configurations for ARIMA: (p,d,q) --> (0,1,0) & (5,1,4)

model = ARIMA(data['Adj Close'], order=(0,1,0))
arima_010_results = model.fit()
arima_010_residuals = arima_010_results.resid

plt.plot(arima_010_residuals)
plt.xlabel('Time')
plt.ylabel('Residuals')
plt.title('Residuals - ARIMA(0,1,0)')

In [ ]:
model = ARIMA(data['Adj Close'], order=(5,1,4))
arima_514_results = model.fit()
arima_514_residuals = arima_514_results.resid

plt.plot(arima_514_residuals)
plt.xlabel('Time')
plt.ylabel('Residuals')
plt.title('Residuals - ARIMA(5,1,4)')

## Prediction and model evaluation

In [ ]:
def mean_absolute_percentage_error(y_pred, y_true):
    y_pred, y_true = np.array(y_pred), np.array(y_true)
    nonzero_indices = y_true != 0
    if not np.any(nonzero_indices):
        return np.inf
    return np.mean(np.abs((y_pred[nonzero_indices] - y_true[nonzero_indices]) / y_true[nonzero_indices] )) * 100


train_size = int(len(data)*0.8)
train_sample, test_sample = data['Adj Close'][:train_size], data['Adj Close'][train_size:]

In [ ]:
def rolling_forecast_1_day_ahead(train, test, order=(0,0,0)):
    history = list(train)
    predictions = []

    for t in range(len(test)):
        model = ARIMA(history, order=order)
        model_fit = model.fit()
        output = model_fit.forecast(steps=1)
        pred = output[-1]
    
        predictions.append(pred)
    
        history.append(test.iloc[t])

    return predictions

predictions_1_day_ahead_010 = rolling_forecast_1_day_ahead(train_sample, test_sample, (0,1,0))
predictions_1_day_ahead_111 = rolling_forecast_1_day_ahead(train_sample, test_sample, (1,1,1))

In [ ]:
mape_010_1_day = mean_absolute_percentage_error(predictions_1_day_ahead_010, test_sample)
rmse_010_1_day = mean_squared_error(test_sample, predictions_1_day_ahead_010)
mae_010_1_day = mean_absolute_error(test_sample, predictions_1_day_ahead_010)
r2_010_1_day = r2_score(test_sample, predictions_1_day_ahead_010)

mape_111_1_day = mean_absolute_percentage_error(predictions_1_day_ahead_111, test_sample)
rmse_111_1_day = mean_squared_error(test_sample, predictions_1_day_ahead_111)
mae_111_1_day = mean_absolute_error(test_sample, predictions_1_day_ahead_111)
r2_111_1_day = r2_score(test_sample, predictions_1_day_ahead_111)

print('Evaluation Metrics (1 day ahead) - ARIMA(0, 1, 0)\n')
print(f"RMSE: {rmse_010_1_day}")
print(f"MAPE: {mape_010_1_day}")
print(f"MAE: {mae_010_1_day}")
print(f"R^2: {r2_010_1_day}")

print('\n\n')
print('Evaluation Metrics (1 day ahead) - ARIMA(1, 1, 1)\n')
print(f"RMSE: {rmse_111_1_day}")
print(f"MAPE: {mape_111_1_day}")
print(f"MAE: {mae_111_1_day}")
print(f"R^2: {r2_111_1_day}")

In [ ]:
def rolling_forecast_7_days_ahead(train, test, order=(0,0,0)):
    history = list(train)
    predictions = []

    for t in range(len(test) - 6):
        model = ARIMA(history, order=order)
        model_fit = model.fit()
        output = model_fit.forecast(steps=7)
        pred = output[-1]
    
        predictions.append(pred)
    
        history.append(test.iloc[t])

    return predictions

predictions_7_days_ahead_010 = rolling_forecast_7_days_ahead(train_sample, test_sample, (0,1,0))
predictions_7_days_ahead_111 = rolling_forecast_7_days_ahead(train_sample, test_sample, (1,1,1))

In [ ]:
mape_010_7_days = mean_absolute_percentage_error(predictions_7_days_ahead_010, test_sample[6:])
rmse_010_7_days = np.sqrt(mean_squared_error(test_sample[6:], predictions_7_days_ahead_010))
mae_010_7_days = mean_absolute_error(test_sample[6:], predictions_7_days_ahead_010)
r2_010_7_days = r2_score(test_sample[6:], predictions_7_days_ahead_010)

mape_111_7_days = mean_absolute_percentage_error(predictions_7_days_ahead_111, test_sample[6:])
rmse_111_7_days = np.sqrt(mean_squared_error(test_sample[6:], predictions_7_days_ahead_111))
mae_111_7_days = mean_absolute_error(test_sample[6:], predictions_7_days_ahead_111)
r2_111_7_days = r2_score(test_sample[6:], predictions_7_days_ahead_111)

print('Evaluation Metrics (7 days ahead) - ARIMA(0, 1, 0)\n')
print(f"RMSE: {rmse_010_7_days}")
print(f"MAPE: {mape_010_7_days}")
print(f"MAE: {mae_010_7_days}")
print(f"R^2: {r2_010_7_days}")

print('\n\n')
print('Evaluation Metrics (7 days ahead) - ARIMA(1, 1, 1)\n')
print(f"RMSE: {rmse_111_7_days}")
print(f"MAPE: {mape_111_7_days}")
print(f"MAE: {mae_111_7_days}")
print(f"R^2: {r2_111_7_days}")

In [ ]:
def rolling_forecast_30_days_ahead(train, test, order=(0,0,0)):
    history = list(train)
    predictions = []

    for t in range(len(test) - 29):
        model = ARIMA(history, order=order)
        model_fit = model.fit()
        output = model_fit.forecast(steps=30)
        pred = output[-1]
    
        predictions.append(pred)
    
        history.append(test.iloc[t])

    return predictions

predictions_30_days_ahead_010 = rolling_forecast_30_days_ahead(train_sample, test_sample, (0,1,0))
predictions_30_days_ahead_111 = rolling_forecast_30_days_ahead(train_sample, test_sample, (1,1,1))

In [ ]:
mape_010_30_days = mean_absolute_percentage_error(predictions_30_days_ahead_010, test_sample[29:])
rmse_010_30_days = np.sqrt(mean_squared_error(test_sample[29:], predictions_30_days_ahead_010))
mae_010_30_days = mean_absolute_error(test_sample[29:], predictions_30_days_ahead_010)
r2_010_30_days = r2_score(test_sample[29:], predictions_30_days_ahead_010)

mape_111_30_days = mean_absolute_percentage_error(predictions_30_days_ahead_111, test_sample[29:])
rmse_111_30_days = np.sqrt(mean_squared_error(test_sample[29:], predictions_30_days_ahead_111))
mae_111_30_days = mean_absolute_error(test_sample[29:], predictions_30_days_ahead_111)
r2_111_30_days = r2_score(test_sample[29:], predictions_30_days_ahead_111)

print('Evaluation Metrics (30 days ahead) - ARIMA(0, 1, 0)\n')
print(f"RMSE: {rmse_010_30_days}")
print(f"MAPE: {mape_010_30_days}")
print(f"MAE: {mae_010_30_days}")
print(f"R^2: {r2_010_30_days}")

print('\n\n')
print('Evaluation Metrics (30 days ahead) - ARIMA(1, 1, 1)\n')
print(f"RMSE: {rmse_111_30_days}")
print(f"MAPE: {mape_111_30_days}")
print(f"MAE: {mae_111_30_days}")
print(f"R^2: {r2_111_30_days}")

# Normalization

In [ ]:
def normalization(df):
    for column in df.select_dtypes(include=['number']).columns:
        df[column] = (df[column] - df[column].min())/(df[column].max() - df[column].min()) 
    return df

data = normalization(data)

data.head()

# Time-Series Neural Network

- Optimizator: Adam
- Batch size: 32
- Look-back period: [3, 8, 14, 30]
- Kernel size (following LB period): [2, 3, 5, 7]
- Number of kernels and number of neurons for time attention layer and hidden layer: [32, 64, 128]
- Learning rate: [0.01, 0.001, 0.0001]
- Dropout: [0.1;0.5]
- Activation function: []

## Further data preparation

In [ ]:
torch.manual_seed(42)
np.random.seed(42)

In [ ]:
features = data[['Open', 'High', 'Low', 'Volume']].values
labels = data[['Adj Close']].values

x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
x_tuning_train, x_val, y_tuning_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [ ]:
class TSdataset(Dataset):
    def __init__(self, features, labels, sequence_length, prediction_horizon):
      self.features = features  
      self.labels = labels  
      self.sequence_length = sequence_length  
      self.prediction_horizon = prediction_horizon

    def __len__(self):
        return len(self.features) - self.sequence_length - self.prediction_horizon + 1

    def __getitem__(self, idx):
        return (self.features[idx:idx + self.sequence_length], self.labels[idx + self.sequence_length + self.prediction_horizon - 1])

def create_dataloader(features_train, labels_train, features_val, labels_val, sequence_length, prediction_horizon):
    train_dataset = TSdataset(features_train, labels_train, sequence_length, prediction_horizon)
    val_dataset = TSdataset(features_val, labels_val, sequence_length, prediction_horizon)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    return train_loader, val_loader

## Fine Tuning without Time Attention

In [ ]:
class TNN(nn.Module):
    def __init__(self, input_size, kernel_output_size, kernel_size, hidden_size, output_size, activation_function, dropout_rate):
        super().__init__()
        self.kernel_filter = nn.Conv1d(in_channels=input_size, out_channels=kernel_output_size, kernel_size=kernel_size)

        self.fc1 = nn.Linear(kernel_output_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

        self.activation_function = activation_function
        self.dropout = nn.Dropout(dropout_rate)


    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.kernel_filter(x)
        x = torch.relu(x)

        x = torch.sum(x, dim=2)

        x = self.fc1(x)
        x = self.activation_function(x)
        x = self.dropout(x)
        x = self.fc2(x)

        return x

def train_and_validate(model, train_loader, val_loader, optimizer, epochs):
    criterion = nn.MSELoss()
    for epoch in range(epochs):
        model.train()
        for features, labels in train_loader:
            features, labels = features.float(), labels.float().unsqueeze(1)
            outputs = model(features)
            loss = criterion(outputs, labels.squeeze(1))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        val_predictions, val_targets = [], []
        with torch.no_grad():
            for features, labels in val_loader:
                features, labels = features.float(), labels.float().unsqueeze(1)
                outputs = model(features)
                val_predictions.extend(outputs.numpy())
                val_targets.extend(labels.numpy())
        
        val_rmse = np.sqrt(np.mean((np.array(val_targets) - np.array(val_predictions))**2))

    return val_rmse




def get_kernel_size(sequence_length):
    if sequence_length == 3:
        return 2
    elif sequence_length == 8:
        return 3
    elif sequence_length == 14:
        return 5
    elif sequence_length == 30:
        return 7
    else:
        return 11



def grid_search(hidden_sizes, activations, dropouts, sequence_lengths, learning_rates):
    best_val_rmse = float('inf')
    best_params = None

    activation_map = {'relu': torch.relu, 'sigmoid': torch.sigmoid, 'tanh': torch.tanh}
    horizons = [1, 7, 30]

    for hidden_size in hidden_sizes:
        for activation_func in activations:
            for dropout_rate in dropouts:
                for seq_len in sequence_lengths:
                    for lr in learning_rates:
                        kernel_size = get_kernel_size(seq_len)
                        print(f"Testing with hidden_size={hidden_size}, activation={activation_func}, sequence_length={seq_len}, kernel_size={kernel_size}, lr={lr}")
                        
                        model = TNN(input_size=4, kernel_output_size=32, kernel_size=kernel_size, hidden_size=hidden_size, output_size=1, activation_function=activation_map[activation_func], dropout_rate=dropout_rate)
                        optimizer = optim.Adam(model.parameters(), lr=lr)

                        avg_rmse = 0
                        for horizon in horizons:
                            train_loader, val_loader = create_dataloader(x_tuning_train, y_tuning_train, x_val, y_val, seq_len, horizon)
                            horizon_rmse = train_and_validate(model, train_loader, val_loader, optimizer, epochs=100)
                            avg_rmse += horizon_rmse
                        avg_rmse /= len(horizons)
                        print(f"Average rmse across horizons: {avg_rmse:.8f}")

                        if avg_rmse < best_val_rmse:
                            best_val_rmse = avg_rmse
                            best_params = (hidden_size, activation_func, dropout_rate, seq_len, kernel_size, lr)


    print(f'Best Parameters: hidden_size: {best_params[0]}, activation_func: {best_params[1]}, dropout_rate: {best_params[2]}, seq_len: {best_params[3]}, kernel_size: {best_params[4]}, lr: {best_params[5]}')
    print(f'Best RMSE: {best_val_rmse}')

In [ ]:
hidden_sizes = [32, 64, 128]
activations = ['relu', 'sigmoid', 'tanh'] 
dropouts = [0.2, 0.3]
sequence_lengths = [3, 8, 14, 30]
learning_rates = [0.01, 0.001, 0.0001]

grid_search(hidden_sizes, activations, dropouts, sequence_lengths, learning_rates)

## Fine Tuning with Time Attention

In [ ]:
class TimeAttention(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, activation_function):
        super().__init__()
        self.attention_fc1 = nn.Linear(input_size, hidden_size)
        self.attention_fc2 = nn.Linear(hidden_size, output_size)
        self.activation_function = activation_function
        self.attention_softmax = nn.Softmax(dim=1)


    def forward(self, features):
        attention_hidden = self.activation_function(self.attention_fc1(features))
        attention_scores = self.attention_fc2(attention_hidden)
        attention_weights = self.attention_softmax(attention_scores)
        weighted_features = attention_weights * features
        return weighted_features

class TNNwithAttention(nn.Module):
    
    def __init__(self, input_size, kernel_output_size, kernel_size, hidden_size, output_size, activation_function, dropout_rate, attention_hidden_size, attention_activation):
        super().__init__()
        self.kernel_filter = nn.Conv1d(in_channels=input_size, out_channels=kernel_output_size, kernel_size=kernel_size)
        self.time_attention = TimeAttention(kernel_output_size, attention_hidden_size, kernel_output_size, activation_function)
        self.fc1 = nn.Linear(kernel_output_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.activation_function = activation_function
        self.dropout = nn.Dropout(dropout_rate)


    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.kernel_filter(x)
        x = torch.relu(x)
        
        x = x.permute(0, 2, 1)
        x = self.time_attention(x)
        x = torch.sum(x, dim=1)
        
        x = self.fc1(x)
        x = self.activation_function(x)
        x = self.dropout(x)
        x = self.fc2(x)

        return x
        

def grid_search_with_time_attention():

    best_val_rmse = float('inf')
    best_params = None
    
    dropout_rate = 0.2
    sequence_length = 3
    learning_rate = 0.01
    kernel_size = 2
    horizons = [1, 7, 30]

    for num_kernels in num_kernels_range:
        for hidden_size in hidden_sizes:
            for attention_hidden_size in attention_hidden_sizes:
                for attention_activation_function in attention_activation_functions:
                    print(f'Testing with num_kernels={num_kernels}, hidden_size={hidden_size}, attention_hidden_size={attention_hidden_size}, attention_activation_function={attention_activation_function}')

                    model = TNNwithAttention(input_size=4, kernel_output_size=num_kernels, kernel_size=kernel_size, hidden_size=hidden_size, output_size=1, activation_function=torch.relu dropout_rate=dropout_rate, attention_hidden_size=attention_hidden_size, attention_activation=attention_activation_function)
                    optimizer = optim.Adam(model.parameters, lr=learning_rate)

                    avg_rmse = 0
                    for horizon in horizons:
                        train_loader,val_loader = create_dataloader(x_tuning_train, y_tuning_train, x_val, y_val, sequence_length, horizon)
                        horizon_rmse = train_and_validate(model, train_loader, val_loader, optimizer, epochs=100)
                        avg_rmse += horizon_rmse

                    avg_rmse /= len(horizons)
                    print(f'Average RMSE across horizons: {avg_rmse}')

                    if avg_rmse < best_val_rmse:
                        best_val_rmse = avg_rmse
                        best_params = (num_kernels, hidden_size, attention_hidden_size, attention_activation_function)

    print(f'Best parameters: num_kernels={best_params[0]}, hidden_size={best_params[1]}, attention_hidden_size={best_params[2]}, attention_activation_function={best_params[3]}')
    print(f'Best RMSE: {best_val_rmse}')
                        

In [ ]:
num_kernels_range = [32, 64, 128]
hidden_sizes = [32, 64, 128]
attention_hidden_sizes = [32, 64, 128]
attention_activation_functions = [torch.tanh, torch.relu]

grid_search_with_time_attention(num_kernels_range, hidden_sizes, attention_hidden_sizes, attention_activation_functions)